## Legenda
1 - CORRENDO
2 - ANDANDO
3 - CAINDO

### Import das bibliotecas utilizadas

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Seleciona acelX, acelY, acelZ e atividade

In [4]:
df_andando  = pd.read_csv("C:\\classificador-de-padroes-de-mobilidade\\dataset\\raw\\Andando-PausasNãoMarcadas.csv")
df_correndo = pd.read_excel("C:\\classificador-de-padroes-de-mobilidade\\dataset\\raw\\Todas_as_Corridas.xlsx")
df_caindo   = pd.read_excel("C:\\classificador-de-padroes-de-mobilidade\\dataset\\raw\\Todas_as_Quedas.xlsx")

colunas_selecionadas = ['acelX', 'acelY', 'acelZ', 'idTipoMovimento']

df_andando   = df_andando[colunas_selecionadas]
df_caindo    = df_caindo[colunas_selecionadas]
df_correndo  = df_correndo[colunas_selecionadas]

# Selecionando somente as 500 primeiras linhas de cada dataset
df_andando   = df_andando.head(500)
df_caindo    = df_caindo.head(500)
df_correndo  = df_correndo.head(500)

### Adiciona cálculo da magnitude da aceleração

In [5]:
df_andando['MAGNITUDE_ACEL']  = np.sqrt(df_andando['acelX']**2 + df_andando['acelY']**2 + df_andando['acelZ']**2)
df_caindo['MAGNITUDE_ACEL']   = np.sqrt(df_caindo['acelX']**2 + df_caindo['acelY']**2 + df_caindo['acelZ']**2)
df_correndo['MAGNITUDE_ACEL'] = np.sqrt(df_correndo['acelX']**2 + df_correndo['acelY']**2 + df_correndo['acelZ']**2)


### Detectando limiares máximos e mínimos de cada dataframe

In [6]:
vl_min_andando  = df_andando['MAGNITUDE_ACEL'].min()
vl_max_andando  = df_andando['MAGNITUDE_ACEL'].max()

vl_min_correndo = df_correndo['MAGNITUDE_ACEL'].min()
vl_max_correndo = df_correndo['MAGNITUDE_ACEL'].max()

vl_min_caindo   = df_caindo['MAGNITUDE_ACEL'].min()
vl_max_caindo   = df_caindo['MAGNITUDE_ACEL'].max()


# Formatando a saída
print(f"Valores para a condição 'Andando':")
print(f"- Mínimo: {vl_min_andando:.2f}")
print(f"- Máximo: {vl_max_andando:.2f}")

print(f"\nValores para a condição 'Correndo':")
print(f"- Mínimo: {vl_min_correndo:.2f}")
print(f"- Máximo: {vl_max_correndo:.2f}")

print(f"\nValores para a condição 'Caindo':")
print(f"- Mínimo: {vl_min_caindo:.2f}")
print(f"- Máximo: {vl_max_caindo:.2f}")


Valores para a condição 'Andando':
- Mínimo: 2621.89
- Máximo: 6936.07

Valores para a condição 'Correndo':
- Mínimo: 607.43
- Máximo: 18624.61

Valores para a condição 'Caindo':
- Mínimo: 1252.15
- Máximo: 22896.89


### Calculando valor de média e desvio padrão para a magnitude

In [7]:
media_andando    = df_andando['MAGNITUDE_ACEL'].mean()
media_correndo   = df_correndo['MAGNITUDE_ACEL'].mean()
media_caindo     = df_caindo['MAGNITUDE_ACEL'].mean()

dev_pad_andando  = df_andando['MAGNITUDE_ACEL'].std()
dev_pad_correndo = df_correndo['MAGNITUDE_ACEL'].std()
dev_pad_caindo   = df_caindo['MAGNITUDE_ACEL'].std()

#Formatando a saída
print(f"Média e Desvio Padrão da Magnitude da Aceleração:")
print(f"Andando: Média = {media_andando:.2f}, Desvio Padrão = {dev_pad_andando:.2f}")
print(f"Correndo: Média = {media_correndo:.2f}, Desvio Padrão = {dev_pad_correndo:.2f}")
print(f"Caindo: Média = {media_caindo:.2f}, Desvio Padrão = {dev_pad_caindo:.2f}")

Média e Desvio Padrão da Magnitude da Aceleração:
Andando: Média = 3873.58, Desvio Padrão = 670.36
Correndo: Média = 5423.58, Desvio Padrão = 4367.87
Caindo: Média = 4718.67, Desvio Padrão = 1975.92


### Concatena e embaralha os datasets

In [57]:
df_completo    = pd.concat([df_andando, df_caindo, df_correndo])
df_embaralhado = df_completo.sample(frac=1).reset_index(drop=True)

### Classificação

In [70]:
media_df_embaralhado = df_embaralhado['MAGNITUDE_ACEL'].mean()
des_pad_df_embaralhado = df_embaralhado['MAGNITUDE_ACEL'].std()

def classifica_atividade(magnitude, mean, std):
    if magnitude < mean - std:
        return 2
    elif mean - std <= magnitude < mean + std:
        return 1
    elif magnitude >= mean + std:
        return 3
    else:
        return 0

df_embaralhado['ATIVIDADE'] = df_embaralhado['MAGNITUDE_ACEL'].apply(lambda x: classifica_atividade(x, media_df_embaralhado, des_pad_df_embaralhado))

### Acurácia

In [71]:
from sklearn.metrics import precision_score

# Verificar se as colunas de atividades preditas e reais existem
if 'ATIVIDADE' in df_embaralhado.columns and 'idTipoMovimento' in df_embaralhado.columns:
    # Contar o número total de previsões
    total = len(df_embaralhado)
    
    # Contar o número de previsões corretas
    corretas = (df_embaralhado['ATIVIDADE'] == df_embaralhado['idTipoMovimento']).sum()
    
    # Calcular a acurácia
    acuracia = corretas / total
    print(f"Acurácia: {acuracia:.2f}")
else:
    print("Colunas 'ATIVIDADE' ou 'ATIVIDADE_REAL' não encontradas no DataFrame.")


Acurácia: 0.20


### Precisão

In [72]:
# Verificar se as colunas de atividades preditas e reais existem
if 'ATIVIDADE' in df_embaralhado.columns and 'idTipoMovimento' in df_embaralhado.columns:
    # Calcular a precisão para cada classe
    precisao = {}
    classes = df_embaralhado['ATIVIDADE'].unique()
    
    for classe in classes:
        # Calcular precisão para a classe atual
        y_true = (df_embaralhado['idTipoMovimento'] == classe).astype(int)
        y_pred = (df_embaralhado['ATIVIDADE'] == classe).astype(int)
        
        precision = precision_score(y_true, y_pred)
        precisao[classe] = precision
    
    # Exibir precisão para cada classe
    for classe, prec in precisao.items():
        print(f"Precisão para a classe {classe}: {prec:.2f}")
else:
    print("Colunas 'ATIVIDADE' ou 'ATIVIDADE_REAL' não encontradas no DataFrame.")

Precisão para a classe 1: 0.22
Precisão para a classe 3: 0.18
Precisão para a classe 2: 0.00
